In [1]:
1+1

2

In [1]:
## rag with mongodb - data ingestion , retrival and generation pipeline 
from dotenv import load_dotenv
import os

load_dotenv()


True

In [2]:
from openai import OpenAI

#initialize the client
client=OpenAI()

model = "text-embedding-3-large"

## define the function that will generate the embeddings
def get_embeddings(text, input_type="document"):
    response = client.embeddings.create(
        model=model,
        input=text
    )
    return response.data[0].embedding

In [10]:
get_embeddings("what is rag")

[-0.046387769281864166,
 -0.049214523285627365,
 -0.017413534224033356,
 0.029409119859337807,
 -0.03602300211787224,
 0.010382886976003647,
 -0.025730716064572334,
 0.002670467831194401,
 -0.016498463228344917,
 0.027470257133245468,
 -0.002915090648457408,
 -0.023973053321242332,
 0.030170170590281487,
 -0.029191678389906883,
 -0.024027414619922638,
 0.02274087816476822,
 -0.023683130741119385,
 -0.009087291546165943,
 0.01764003559947014,
 -0.0002174426626879722,
 0.03294256329536438,
 -0.002767863916233182,
 0.0014031847240403295,
 0.034029778093099594,
 0.00014439552614931017,
 0.014505238272249699,
 0.001407714793458581,
 0.02973528392612934,
 -0.05555660277605057,
 0.014595839194953442,
 0.06776963174343109,
 0.0438871793448925,
 0.01429685577750206,
 0.0013816669816151261,
 -0.038233667612075806,
 -0.014586779288947582,
 0.01390727050602436,
 0.046387769281864166,
 -0.002389604225754738,
 -0.012611675076186657,
 0.006242416799068451,
 0.017395412549376488,
 -0.05860079824924469

In [17]:
#data ingestion
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

#load the pdf now in the code
loader = PyPDFLoader("https://investors.mongodb.com/node/12236/pdf")
data = loader.load()

## now split the data into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)
documents = text_splitter.split_documents(data)


In [18]:
documents

[Document(metadata={'producer': 'West Corporation using ABCpdf', 'creator': 'PyPDF', 'creationdate': '2024-05-30T20:06:12+00:00', 'title': 'MongoDB, Inc. Announces First Quarter Fiscal 2025 Financial Results', 'source': 'https://investors.mongodb.com/node/12236/pdf', 'total_pages': 8, 'page': 0, 'page_label': '1'}, page_content='MongoDB, Inc. Announces First Quarter Fiscal 2025 Financial Results\nMay 30, 2024\nFirst Quarter Fiscal 2025 Total Revenue of $450.6 million, up 22% Year-over-Year\nContinued Strong Customer Growth with Over 49,200 Customers as of April 30, 2024\nMongoDB Atlas Revenue up 32% Year-over-Year; 70% of Total Q1 Revenue'),
 Document(metadata={'producer': 'West Corporation using ABCpdf', 'creator': 'PyPDF', 'creationdate': '2024-05-30T20:06:12+00:00', 'title': 'MongoDB, Inc. Announces First Quarter Fiscal 2025 Financial Results', 'source': 'https://investors.mongodb.com/node/12236/pdf', 'total_pages': 8, 'page': 0, 'page_label': '1'}, page_content='NEW YORK, May 30, 2

In [20]:
## now till this point the document data is converted into chunks. now we have to convert this data into vector embeddings and then store them in the vector db.

# Prepare documents for insertion
docs_to_insert = [{
    "text": doc.page_content,
    "embedding": get_embeddings(doc.page_content)
} for doc in documents]

In [21]:
docs_to_insert

[{'text': 'MongoDB, Inc. Announces First Quarter Fiscal 2025 Financial Results\nMay 30, 2024\nFirst Quarter Fiscal 2025 Total Revenue of $450.6 million, up 22% Year-over-Year\nContinued Strong Customer Growth with Over 49,200 Customers as of April 30, 2024\nMongoDB Atlas Revenue up 32% Year-over-Year; 70% of Total Q1 Revenue',
  'embedding': [-0.00868505984544754,
   0.01482553780078888,
   -0.010593496263027191,
   0.03958833962678909,
   0.011062235571444035,
   -0.017557615414261818,
   -0.03693661466240883,
   0.06701625138521194,
   -0.018481699749827385,
   0.05705220624804497,
   0.05528439208865166,
   -0.009193976409733295,
   -0.00673979427665472,
   -0.011879179626703262,
   -0.0484006293118,
   0.024052996188402176,
   -0.028767168521881104,
   0.010472963564097881,
   -0.0008721889462321997,
   -0.026744896546006203,
   -0.032061733305454254,
   0.004473107401281595,
   0.017812073230743408,
   0.045641764998435974,
   -0.016271932050585747,
   -0.01321843359619379,
   0.0

In [ ]:
from pymongo import MongoClient

# Connect to your MongoDB deployment
mongodb_uri = os.getenv("MONGODB_CONNECTION_STRING")
client = MongoClient(mongodb_uri)
collection =  client["rag_db"]["test"]

# Insert documents into the collection
result = collection.insert_many(docs_to_insert)

In [23]:
result

InsertManyResult([ObjectId('696777481ee2a940f8f1b032'), ObjectId('696777481ee2a940f8f1b033'), ObjectId('696777481ee2a940f8f1b034'), ObjectId('696777481ee2a940f8f1b035'), ObjectId('696777481ee2a940f8f1b036'), ObjectId('696777481ee2a940f8f1b037'), ObjectId('696777481ee2a940f8f1b038'), ObjectId('696777481ee2a940f8f1b039'), ObjectId('696777481ee2a940f8f1b03a'), ObjectId('696777481ee2a940f8f1b03b'), ObjectId('696777481ee2a940f8f1b03c'), ObjectId('696777481ee2a940f8f1b03d'), ObjectId('696777481ee2a940f8f1b03e'), ObjectId('696777481ee2a940f8f1b03f'), ObjectId('696777481ee2a940f8f1b040'), ObjectId('696777481ee2a940f8f1b041'), ObjectId('696777481ee2a940f8f1b042'), ObjectId('696777481ee2a940f8f1b043'), ObjectId('696777481ee2a940f8f1b044'), ObjectId('696777481ee2a940f8f1b045'), ObjectId('696777481ee2a940f8f1b046'), ObjectId('696777481ee2a940f8f1b047'), ObjectId('696777481ee2a940f8f1b048'), ObjectId('696777481ee2a940f8f1b049'), ObjectId('696777481ee2a940f8f1b04a'), ObjectId('696777481ee2a940f8f1b0

In [24]:
from pymongo.operations import SearchIndexModel
import time

# Create your index model, then create the search index
index_name="vector_index"
search_index_model = SearchIndexModel(
  definition = {
    "fields": [
      {
        "type": "vector",
        "numDimensions": 3072,
        "path": "embedding",
        "similarity": "cosine"
      }
    ]
  },
  name = index_name,
  type = "vectorSearch"
)
collection.create_search_index(model=search_index_model)

# Wait for initial sync to complete
print("Polling to check if the index is ready. This may take up to a minute.")
predicate=None
if predicate is None:
   predicate = lambda index: index.get("queryable") is True

while True:
   indices = list(collection.list_search_indexes(index_name))
   if len(indices) and predicate(indices[0]):
      break
   time.sleep(5)
print(index_name + " is ready for querying.")

Polling to check if the index is ready. This may take up to a minute.
vector_index is ready for querying.


In [26]:
query_embedding = get_embeddings("AI Technology")
collection.test.aggregate([
  {
    "$vectorSearch": {
      "index": "vector_index",
      "path": "embedding",
      "queryVector":query_embedding,
      "numCandidates":3072 ,
      "limit": 5
    }
  }
])

TypeError: 'Collection' object is not callable. If you meant to call the 'create' method on a 'Database' object it is failing because no such method exists.